In [1]:
import time
import json
import asyncio
import random
from typing import List
from datetime import datetime

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

from helpers.selenium_management import start_driver, open_link, get_wait_element, get_wait_elements, close_driver, get_link_elements, get_links
from helpers.utils import extract_original_link, parse_activity_data, validate_instagram_url, POST_VALUE
from helpers.excel import PyXLWriter

import config
from robot import auth, account_get_post_links, account_send_comment

In [2]:
with open(config.ACCOUNT_LINKS_PATH, "r", encoding="utf-8") as file:
    accounts = file.read()
    accounts = accounts.split('\n')

# Список шаблонов сообщений
with open(config.MESSAGE_TEMPLATES_PATH, "r", encoding="utf-8") as file:
    messages = json.load(file)

# Список шаблонов комментариев
with open(config.COMMENT_TEMPLATES_PATH, "r", encoding="utf-8") as file:
    comments = json.load(file)

# Список данных от аккаунтов
with open(config.AUTH_DATA_PATH, "r", encoding="utf-8") as file:
    auth_data_list = json.load(file)

In [3]:
driver = start_driver()

auth_data = auth_data_list[0]
username = auth_data.get('username')
password = auth_data.get('password')

auth(driver=driver, username=username, password=password)

HTTP-код 200. Страница доступна.


True

In [11]:
def account_get_post_links(driver: webdriver, account_link: str) -> list:
    """
    Парсинг ссылок на посты аккаунта
    """
    open_link(driver, account_link)
    posts_parent_element = get_wait_element(
        driver=driver,
        by=By.XPATH,
        searched_elem='/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[2]/div/div[1]/section/main/div/div[2]',
        delay=15,
        sleep=3,
        attempts=2
    )
    link_elements = get_link_elements(posts_parent_element)
    post_links = get_links(link_elements=link_elements)
    return post_links



In [4]:
account_link = "https://www.instagram.com/derekeh_/"
post_links = account_get_post_links(driver=driver, account_link=account_link)

HTTP-код 200. Страница доступна.


In [73]:
def account_send_comment(driver: webdriver, post_link: str, comment: str) -> bool:
    """
    Отправка комментария к посту
    """
    open_link(driver, post_link)
    
    comment_element = get_wait_element(
        driver=driver,
        by=By.XPATH,
        searched_elem='//textarea[@aria-label="Добавьте комментарий..."]',
        delay=15,
        sleep=3,
        attempts=2
    )
    if comment_element:    
        ActionChains(driver).click(comment_element).send_keys(str(comment)).perform()
        time.sleep(3)
        driver.switch_to.active_element.send_keys(Keys.ENTER)  # Отправляет коммент
        print(f'Написал комментарий к посту: {post_link}')
        return True
    print(f'Не получилось написать комментарий к посту: {post_link}')
    return False



In [5]:
first_post_link = post_links[0]
account_send_comment(driver=driver, post_link=first_post_link, comment=random.choice(comments))

HTTP-код 200. Страница доступна.
Написал комментарий к посту: https://www.instagram.com/derekeh_/p/DD95M70vxlH/


True

In [55]:
comment_element = get_wait_element(
        driver=driver,
        by=By.XPATH,
        searched_elem='//textarea[@aria-label="Добавьте комментарий..."]',
        delay=15,
        sleep=3,
        attempts=2
    )

In [53]:
ActionChains(driver).click_and_hold(comment_element).click().send_keys('wow')

In [62]:
action = ActionChains(driver)
action.click(comment_element)
action.send_keys('wow').perform()
comment_element.send_keys()

In [70]:
random.choice(comments)

'🙌'

In [77]:
message_num = 2
account_counter = 1
accounts = [1, 2, 3, 4, 5, 6, 7,8 ,9]
for i in range(2):
    for account_link in accounts[account_counter:account_counter+message_num]:
        print(account_counter)
        account_counter += 1
        print('c, ', account_counter)
    

1
c,  2
2
c,  3
3
c,  4
4
c,  5
